In [1]:
# import pyscfad
# from pyscfad import gto,dft,scf
import pyscf
from pyscf import gto,dft,scf

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
import numpy as np
import jax.numpy as jnp
import scipy
from ase import Atoms
from ase.io import read
import xcquinox as xce
from functools import partial
from ase.units import Bohr
import os, optax, jax


2024-04-16 12:10:20.077006: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:280] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [44]:
def get_mol(atoms, basis='6-311++G**'):
    pos = atoms.positions
    spec = atoms.get_chemical_symbols()
    mol_input = [[s, p] for s, p in zip(spec, pos)]
    try:
        mol = gto.Mole(atom=mol_input, basis=atoms.info.get('basis',basis),spin=atoms.info.get('spin',0))
    except Exception:
        mol = gto.Mole(atom=mol_input, basis=atoms.info.get('basis','STO-3G'),spin=atoms.info.get('spin',0))
    return mol 

def get_rhos(rho, spin):
    rho0 = rho[0,0]
    drho = rho[0,1:4] + rho[1:4,0]
    tau = 0.5*(rho[1,1] + rho[2,2] + rho[3,3])

    if spin != 0:
        rho0_a = rho0[0]
        rho0_b = rho0[1]
        gamma_a, gamma_b = jnp.einsum('ij,ij->j',drho[:,0],drho[:,0]), jnp.einsum('ij,ij->j',drho[:,1],drho[:,1])              
        gamma_ab = jnp.einsum('ij,ij->j',drho[:,0],drho[:,1])
        tau_a, tau_b = tau
    else:
        rho0_a = rho0_b = rho0*0.5
        gamma_a=gamma_b=gamma_ab= jnp.einsum('ij,ij->j',drho[:],drho[:])*0.25
        tau_a = tau_b = tau*0.5
    return rho0_a, rho0_b, gamma_a, gamma_b, gamma_ab, tau_a, tau_b
    
def get_data_synth(xc_func, n=100):
    def get_rho(s, a):
        c0 = 2*(3*np.pi**2)**(1/3)
        c1 = 3/10*(3*np.pi**2)**(2/3)
        gamma = c0*s
        tau = c1*a+c0**2*s**2/8
        rho = np.zeros([len(a),6])
        rho[:, 1] = gamma
        rho[:,-1] = tau
        rho[:, 0] = 1
        return rho
    
    s_grid = jnp.concatenate([[0],jnp.exp(jnp.linspace(-10,4,n))])
    rho = []
    for s in s_grid:
        if 'MGGA' in xc_func:
            a_grid = jnp.concatenate([jnp.exp(jnp.linspace(jnp.log((s/100)+1e-8),8,n))])
        else:
            a_grid = jnp.array([0])
        rho.append(get_rho(s, a_grid))
        
    rho = jnp.concatenate(rho)
    
    fxc =  dft.numint.libxc.eval_xc(xc_func,rho.T, spin=0)[0]/dft.numint.libxc.eval_xc('LDA_X',rho.T, spin=0)[0] -1
 
    rho = jnp.asarray(rho)
    
    tdrho = xc.get_descriptors(rho[:,0]/2,rho[:,0]/2,(rho[:,1]/2)**2,(rho[:,1]/2)**2,(rho[:,1]/2)**2,rho[:,5]/2,rho[:,5]/2, spin_scaling=True)
    


    tFxc = torch.from_numpy(fxc)
    return tdrho[0], tFxc

def get_data(mol, xc_func ,full=False, enhance_spin=False, localnet=None):
    print('mol: ', mol.atom)
    mf = dft.RKS(mol)
    print(mf)   
    mf.xc = 'PBE'
    mf.grids.level = 1
    mf.kernel()
    ao = mf._numint.eval_ao(mol, mf.grids.coords, deriv=2)
    dm = mf.make_rdm1()
    if len(dm.shape) == 2:
        dm = np.array([0.5*dm, 0.5*dm])
    print('ao.shape', ao.shape)
    if not full:
        mf.grids.coords = coords
        mf.grids.weights = weights
    if localnet.spin_scaling:
        print('spin scaling, indicates exchange network')
        rho_alpha = mf._numint.eval_rho(mol, ao, dm[0], xctype='metaGGA',hermi=True)
        rho_beta = mf._numint.eval_rho(mol, ao, dm[1], xctype='metaGGA',hermi=True)
        fxc_a =  mf._numint.eval_xc(xc_func,(rho_alpha,rho_alpha*0), spin=1)[0]/mf._numint.eval_xc('LDA_X',(rho_alpha,rho_alpha*0), spin=1)[0] -1
        fxc_b =  mf._numint.eval_xc(xc_func,(rho_beta*0,rho_beta), spin=1)[0]/mf._numint.eval_xc('LDA_X',(rho_beta*0,rho_beta), spin=1)[0] -1
        print('fxc with xc_func = {} = {}'.format(fxc_a, xc_func))

        if mol.spin != 0 and sum(mol.nelec)>1:
            rho = jnp.concatenate([rho_alpha, rho_beta])
            fxc = jnp.concatenate([fxc_a, fxc_b])
        else:
            rho = rho_alpha
            fxc = fxc_a
    else:    
        print('no spin scaling, indicates correlation network')
        rho_alpha = mf._numint.eval_rho(mol, ao, dm[0], xctype='metaGGA',hermi=True)
        rho_beta = mf._numint.eval_rho(mol, ao, dm[1], xctype='metaGGA',hermi=True)
        exc = mf._numint.eval_xc(xc_func,(rho_alpha,rho_beta), spin=1)[0]
        print('exc with xc_func = {} = {}'.format(exc, xc_func))
        fxc = exc/mf._numint.eval_xc('LDA_C_PW',(rho_alpha, rho_beta), spin=1)[0] -1
        rho = jnp.stack([rho_alpha,rho_beta], axis=-1)
    
    dm = jnp.array(mf.make_rdm1())
    print('get_data, dm shape = {}'.format(dm.shape))
    ao_eval = jnp.array(mf._numint.eval_ao(mol, mf.grids.coords, deriv=1))
    rho = jnp.einsum('xij,yik,...jk->xy...i', ao_eval, ao_eval, dm)
    print('rho shape', rho.shape)
    if dm.ndim == 3:
        rho_filt = (jnp.sum(rho[0,0],axis=0) > 1e-6)
    else:
        rho_filt = (rho[0,0] > 1e-6)
    print('rho_filt shape:', rho_filt.shape)

    
    mf.converged=True
    tdrho = xc.get_descriptors(*get_rhos(rho, spin=mol.spin), spin_scaling=localnet.spin_scaling, mf=mf, dm=dm)
    print('tdrho shape from get_descriptors', tdrho.shape)
        
    if localnet.spin_scaling:
        if mol.spin != 0 and sum(mol.nelec) > 1:
            #tdrho not returned in a spin-polarized form regardless,
            #but the enhancement factors sampled as polarized, so double
            tdrho = jnp.concatenate([tdrho,tdrho])
            rho_filt = jnp.concatenate([rho_filt]*2)
        elif sum(mol.nelec) == 1:
            pass
    
    tdrho = tdrho[rho_filt]
    # tdrho = tdrho
    print('tdrho[rho_filt], rho_filt shapes:', tdrho.shape, rho_filt.shape)
    print('fxc shape', fxc.shape)
    tFxc = jnp.array(fxc)[rho_filt]
    # tFxc = jnp.array(fxc)
    print('returned tdrho, tFxc shape: ', tdrho.shape, tFxc.shape)
    return tdrho, tFxc

In [45]:
PRETRAIN_LEVEL = 'NONLOCAL'

TRAIN_NET = 'x'

REFERENCE_XC = 'PBE0'

N_HIDDEN = 16
DEPTH = 3
if PRETRAIN_LEVEL == 'GGA':
    if TRAIN_NET == 'x':
        localx = xce.net.eX(n_input=1, n_hidden=N_HIDDEN, use=[1], depth=DEPTH, lob=1.804)
    elif TRAIN_NET == 'c':
        localc = xce.net.eC(n_input=3, n_hidden=N_HIDDEN, use=[2], depth=DEPTH, ueg_limit=True)
elif PRETRAIN_LEVEL == 'MGGA':
    if TRAIN_NET == 'x':
        localx = xce.net.eX(n_input=2, n_hidden=N_HIDDEN, use=[1, 2], depth=DEPTH, ueg_limit=True, lob=1.174)
    elif TRAIN_NET == 'c':
        localc = xce.net.eC(n_input=4, n_hidden=N_HIDDEN, depth=DEPTH, use=[2,3], ueg_limit=True)
elif PRETRAIN_LEVEL == 'NONLOCAL':
    if TRAIN_NET == 'x':
        localx = xce.net.eX(n_input=15, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True, lob=1.174)
    elif TRAIN_NET == 'c':
        #n_input = 4 from base, 12 from NL
        localc = xce.net.eC(n_input=16, n_hidden=N_HIDDEN, depth=DEPTH, ueg_limit=True)

if TRAIN_NET == 'x':
    thislocal = localx
else:
    thislocal = localc
ueg = xce.xc.LDA_X()
xc = xce.xc.eXC(grid_models=[thislocal], heg_mult=True, level= {'GGA':2, 'MGGA':3, 'NONLOCAL':4}[PRETRAIN_LEVEL])

In [46]:
spins = {
    'Al': 1,
    'B' : 1,
    'Li': 1,
    'Na': 1,
    'Si': 2 ,
    'Be':0,
    'C': 2,
    'Cl': 1,
    'F': 1,
    'H': 1,
    'N': 3,
    'O': 2,
    'P': 3,
    'S': 2
}

selection = [2, 113, 25, 18, 11, 17, 114, 121, 101, 0, 20, 26, 29, 67, 28, 110, 125, 10, 115, 89, 105, 50]
try:
    atoms = [read('/home/awills/Documents/Research/ogdpyscf/dpyscf/data/haunschild_g2/g2_97.traj',':')[s] for s in selection]
except:
    atoms = [read('/home/awills/Documents/Research/ogdpyscf/data/haunschild_g2/g2_97.traj',':')[s] for s in selection]
ksr_atoms = atoms
if PRETRAIN_LEVEL=='MGGA':
    ksr_atoms = ksr_atoms[2:]
ksr_atoms = [Atoms('P',info={'spin':3}), Atoms('N', info={'spin':3}), Atoms('H', info={'spin':1}),Atoms('Li', info={'spin':1}), Atoms('O',info={'spin':2}),Atoms('Cl',info={'spin':1}),Atoms('Al',info={'spin':1}), Atoms('S',info={'spin':2})] + ksr_atoms
# ksr_atoms = [Atoms('H',info={'spin':1})]
mols = [get_mol(atoms) for atoms in ksr_atoms]
mols = [i for i in mols if len(i.atom) < 8]
for i in mols:
    print(i, i.atom, len(i.atom))

<pyscf.gto.mole.Mole object at 0x7fe8282a45b0> [['P', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe932338370> [['N', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe93233bd30> [['H', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe93233ae60> [['Li', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe7200e5ba0> [['O', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe7200e6980> [['Cl', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe7200e5540> [['Al', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe7200e5420> [['S', array([0., 0., 0.])]] 1
<pyscf.gto.mole.Mole object at 0x7fe7200e74f0> [['H', array([0.      , 0.      , 0.371395])], ['H', array([ 0.      ,  0.      , -0.371395])]] 2
<pyscf.gto.mole.Mole object at 0x7fe7200e7070> [['N', array([0.      , 0.      , 0.549396])], ['N', array([ 0.      ,  0.      , -0.549396])]] 2
<pyscf.gto.mole.Mole object at 0x7fe7200e7550> [['Li', array([ 0.      ,  0.      ,

In [47]:
# data = [get_data(mol, xc_func=ref, full=i<14) for i,mol in enumerate(mols)]
ref = 'PBE0'
data = [get_data(mol, xc_func=ref,full=True, localnet=thislocal) for i,mol in enumerate(mols)]
# 
# data = [get_data_synth(ref, 100)]
tdrho = jnp.concatenate([d[0] for d in data])
tFxc = jnp.concatenate([d[1] for d in data])

mol:  [['P', array([0., 0., 0.])]]
ROKS object of <class 'pyscf.dft.roks.ROKS'>


Initialize <pyscf.gto.mole.Mole object at 0x7fe8282a45b0> in ROKS object of <class 'pyscf.dft.roks.ROKS'>


converged SCF energy = -341.103975305829
ao.shape (10, 6320, 30)
spin scaling, indicates exchange network
fxc with xc_func = [ 0.35227731  0.35296584  0.33914136 ... -0.24267484 -0.24267484
 -0.24267484] = PBE0
get_data, dm shape = (2, 30, 30)
rho shape (4, 4, 2, 6320)
rho_filt shape: (6320,)
Shape mo_coeff present, will pad vele_mat
Initial vele_mat shape:  (30, 30, 6320)
shape_mo_coeff shape:  (30, 30)
Will try to pad to:  [(0, 0), (0, 0), (0, 0)]
get_vele_mat, returned shape: (30, 30, 6320)
get_vele_mat, points shape: (6320, 3)
Evaluating MGGA Data; shapes: 
grid.coords = (6320, 3)
generated ao_data = (20, 6320, 30)
rdm1 input shape: (2, 30, 30)
ao_data shape will be padded to match rdm1
paddeds ao_data = (20, 6320, 30)
nl_4, descr5 shape: (2, 12, 6320)
descr shape:  (6320, 6)
descr5 shape:  (6320, 12)
tdrho shape from get_descriptors (6320, 18)
tdrho[rho_filt], rho_filt shapes: (10940, 18) (12640,)
fxc shape (12640,)
returned tdrho, tFxc shape:  (10940, 18) (10940,)
mol:  [['N', ar

Initialize <pyscf.gto.mole.Mole object at 0x7fe932338370> in ROKS object of <class 'pyscf.dft.roks.ROKS'>


ROKS object of <class 'pyscf.dft.roks.ROKS'>
converged SCF energy = -54.5278183883328
ao.shape (10, 5184, 22)
spin scaling, indicates exchange network
fxc with xc_func = [ 0.35208183  0.35288518  0.35299592 ... -0.2364243  -0.2364243
 -0.2364243 ] = PBE0
get_data, dm shape = (2, 22, 22)
rho shape (4, 4, 2, 5184)
rho_filt shape: (5184,)
Shape mo_coeff present, will pad vele_mat
Initial vele_mat shape:  (22, 22, 5184)
shape_mo_coeff shape:  (22, 22)
Will try to pad to:  [(0, 0), (0, 0), (0, 0)]
get_vele_mat, returned shape: (22, 22, 5184)
get_vele_mat, points shape: (5184, 3)
Evaluating MGGA Data; shapes: 

...

RKS object of <class 'pyscf.dft.rks.RKS'>
converged SCF energy = -40.459821486401
ao.shape (10, 13936, 50)
spin scaling, indicates exchange network
fxc with xc_func = [0.352642   0.35294844 0.35299903 ... 0.35299824 0.35299824 0.35299824] = PBE0
get_data, dm shape = (50, 50)
rho shape (4, 4, 13936)
rho_filt shape: (13936,)
Shape mo_coeff present, will pad vele_mat
Initial vele_mat shape:  (50, 50, 13936)
shape_mo_coeff shape:  (50, 50)
Will try to pad to:  [(0, 0), (0, 0), (0, 0)]
get_vele_mat, returned shape: (50, 50, 13936)
get_vele_mat, points shape: (13936, 3)
Evaluating MGGA Data; shapes: 
grid.coords = (13936, 3)
generated ao_data = (20, 13936, 50)
rdm1 input shape: (50, 50)
paddeds ao_data = (20, 13936, 50)
nl_4, descr5 shape: (12, 13936)
descr shape:  (13936, 6)
descr5 shape:  (13936, 12)
tdrho shape from get_descriptors (13936, 18)
tdrho[rho_filt], rho_filt shapes: (12616, 18) (13936,)
fxc shape (13936,)
returned tdrho, tFxc shape:  (12616, 18) (12616,)
mol:  [['C', array([0. 

Initialize <pyscf.gto.mole.Mole object at 0x7fe7182447f0> in RKS object of <class 'pyscf.dft.rks.RKS'>


RKS object of <class 'pyscf.dft.rks.RKS'>
converged SCF energy = -39.0756147483525
ao.shape (10, 9656, 36)
spin scaling, indicates exchange network
fxc with xc_func = [ 0.35266567  0.35299597  0.35296276 ... -0.08131804 -0.08131804
 -0.08131804] = PBE0
get_data, dm shape = (36, 36)
rho shape (4, 4, 9656)
rho_filt shape: (9656,)
Shape mo_coeff present, will pad vele_mat
Initial vele_mat shape:  (36, 36, 9656)
shape_mo_coeff shape:  (36, 36)
Will try to pad to:  [(0, 0), (0, 0), (0, 0)]
get_vele_mat, returned shape: (36, 36, 9656)
get_vele_mat, points shape: (9656, 3)
Evaluating MGGA Data; shapes: 
grid.coords = (9656, 3)
generated ao_data = (20, 9656, 36)
rdm1 input shape: (36, 36)
paddeds ao_data = (20, 9656, 36)
nl_4, descr5 shape: (12, 9656)
descr shape:  (9656, 6)
descr5 shape:  (9656, 12)
tdrho shape from get_descriptors (9656, 18)
tdrho[rho_filt], rho_filt shapes: (8643, 18) (9656,)
fxc shape (9656,)
returned tdrho, tFxc shape:  (8643, 18) (8643,)
mol:  [['Si', array([0., 0., 0.])

Initialize <pyscf.gto.mole.Mole object at 0x7fe718246920> in RKS object of <class 'pyscf.dft.rks.RKS'>


RKS object of <class 'pyscf.dft.rks.RKS'>
converged SCF energy = -291.719272437817
ao.shape (10, 15256, 58)
spin scaling, indicates exchange network
fxc with xc_func = [0.35267139 0.35295005 0.35299895 ... 0.35299876 0.35299876 0.35299876] = PBE0
get_data, dm shape = (58, 58)
rho shape (4, 4, 15256)
rho_filt shape: (15256,)
Shape mo_coeff present, will pad vele_mat
Initial vele_mat shape:  (58, 58, 15256)
shape_mo_coeff shape:  (58, 58)
Will try to pad to:  [(0, 0), (0, 0), (0, 0)]
get_vele_mat, returned shape: (58, 58, 15256)
get_vele_mat, points shape: (15256, 3)
Evaluating MGGA Data; shapes: 
grid.coords = (15256, 3)
generated ao_data = (20, 15256, 58)
rdm1 input shape: (58, 58)
paddeds ao_data = (20, 15256, 58)
nl_4, descr5 shape: (12, 15256)
descr shape:  (15256, 6)
descr5 shape:  (15256, 12)
tdrho shape from get_descriptors (15256, 18)
tdrho[rho_filt], rho_filt shapes: (13972, 18) (15256,)
fxc shape (15256,)
returned tdrho, tFxc shape:  (13972, 18) (13972,)


In [48]:
nan_filt = ~jnp.any((tdrho != tdrho),axis=-1)

tFxc = tFxc[nan_filt]
tdrho = tdrho[nan_filt,:]

In [49]:
tdrho[::].shape, tdrho[::], tFxc.shape, tFxc

((271339, 18),
 Array([[-3.32705354e+00, -5.19955880e+00,  2.23826797e+00, ...,
         -1.18654557e-04,  5.71783976e-02,  2.61770357e-02],
        [-3.81214229e+00, -6.36530914e+00,  2.88661268e+00, ...,
         -2.06414901e-07,  1.59282231e-02,  1.04687540e-02],
        [-2.97702168e+00, -4.38391796e+00,  1.80625230e+00, ...,
         -4.13498366e-02,  2.45943690e-01,  4.99661002e-02],
        ...,
        [-4.21483075e+00, -4.21483075e+00,  3.09331731e+00, ...,
         -2.39755074e-04,  6.09549468e-02,  2.73448094e-02],
        [-4.25518750e+00, -4.25518750e+00,  3.15236019e+00, ...,
         -3.29185903e-04,  3.78240484e-02,  3.61891725e-03],
        [-4.31527657e+00, -4.31527657e+00,  3.09061936e+00, ...,
          8.22764504e-06,  7.12290327e-02,  6.34636718e-02]],      dtype=float64),
 (271339,),
 Array([0.32321715, 0.34550121, 0.27866855, ..., 0.3483771 , 0.3489568 ,
        0.34849784], dtype=float64))

In [50]:
cpus = jax.devices(backend='cpu')

In [51]:
PRINT_EVERY=200
scheduler = optax.exponential_decay(init_value = 1e-2, transition_begin=50, transition_steps=1000, decay_rate=0.9)
# optimizer = optax.adam(learning_rate = 1e-2)
optimizer = optax.adam(learning_rate = scheduler)

class PT_E_Loss():

    def __call__(self, model, inp, ref):

        pred = jax.vmap(model.net)(inp)[:, 0]

        err = pred-ref

        return jnp.mean(jnp.square(err))

trainer = xce.train.xcTrainer(model=thislocal, optim=optimizer, steps=100000, loss = PT_E_Loss(), do_jit=True)
if TRAIN_NET == 'x':
    inp = [tdrho[:, trainer.model.use]]
else:
    inp = [tdrho]
with jax.default_device(cpus[0]):
    newm = trainer(1, trainer.model, inp, [tFxc])
        

# for epoch in range(100000):
#     total_loss = 0
#     results = thislocal(tdrho[::])
#     loss = eloss(results, tFxc[::])
#     total_loss += loss.item()
#     loss.backward()

#     optimizer.step()
#     optimizer.zero_grad()
#     if epoch%PRINT_EVERY==0:
#         print('total loss {:.12f}'.format(total_loss))
        


Epoch 0
Epoch 0 :: Batch 0/1
Batch Loss = 86392122.6007444
0, epoch_train_loss=86392122.6007444
Epoch 1
Epoch 1 :: Batch 0/1
Batch Loss = 120599456.2417951
1, epoch_train_loss=120599456.2417951
Epoch 2
Epoch 2 :: Batch 0/1
Batch Loss = 45657491.75941516
2, epoch_train_loss=45657491.75941516
Epoch 3
Epoch 3 :: Batch 0/1
Batch Loss = 69765.50179416063
3, epoch_train_loss=69765.50179416063
Epoch 4
Epoch 4 :: Batch 0/1
Batch Loss = 23307120.18519023
4, epoch_train_loss=23307120.18519023
Epoch 5
Epoch 5 :: Batch 0/1
Batch Loss = 36338847.682660595
5, epoch_train_loss=36338847.682660595
Epoch 6
Epoch 6 :: Batch 0/1
Batch Loss = 21355337.450568184
6, epoch_train_loss=21355337.450568184
Epoch 7
Epoch 7 :: Batch 0/1
Batch Loss = 5606432.037689496
7, epoch_train_loss=5606432.037689496
Epoch 8
Epoch 8 :: Batch 0/1
Batch Loss = 31707.353123202374
8, epoch_train_loss=31707.353123202374
Epoch 9
Epoch 9 :: Batch 0/1
Batch Loss = 2881351.5179388854
9, epoch_train_loss=2881351.5179388854
Epoch 10
Epoch

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe96491f9d0>>
Traceback (most recent call last):
  File "/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
jax.vmap(thislocal.net)(tdrho[:, [1,2]])[:, 0].shape

In [ ]:
jax.vmap(trainer.model.net)(tdrho[:, [1,2]])